In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Forecasting using FBProphet and Vertex AI custom container deployment
<table align="left">
   <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/SDK_FBProphet_Forecasting_Online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
    <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/SDK_FBProphet_Forecasting_Online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
   </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/SDK_FBProphet_Forecasting_Online.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This tutorial walks you through building a custom container to serve a facebook prophet model on Vertex AI. You use the FastAPI Python web server framework to create a prediction and health check endpoint. This notebook is a modified example on serving a scikit-learn model on Vertex AI Predictions that can be found [here](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_Custom_Container_Prediction.ipynb).

The facebook-prophet model that you build in this notebook is referred from an [article on testdriven.io: Deploying and Hosting a Machine Learning Model with FastAPI and Heroku](https://testdriven.io/blog/fastapi-machine-learning/).


### Objective

The objective of this notebook is to create, deploy and serve a custom forecasting model on Vertex AI. This notebook focuses more on deploying the model than on the design of the model itself. 

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Models
- Vertex AI Endpoints

The steps performed include:
- Train a model locally that forecasts sales for the number of days passed into the model prediction function
- Train a model that uses both sales and weather data
- Save the models
- Build a FastAPI server to handle predictions and health checks
- Build a custom container with model artifacts
- Upload and deploy the custom container to Vertex AI Predictions



### Dataset

This example uses historical liquor sales data in Iowa from a public dataset in BigQuery: `bigquery-public-data:iowa_liquor_sales.sales` to forecast sales for a given time period in future.

Each instance includes 2 features, a datestamp and the adjusted sales for that time period.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing) and [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**If you are using Colab**, docker related steps are skipped as Colab doesn't fully support docker yet. 

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* Docker
* Git
* Google Cloud SDK (gcloud)
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.


## Install additional packages

Install additional package dependencies not installed in your notebook environment, such as NumPy, pystan, pandas, plotly, fbprophet, FastAPI, Uvicorn, and joblib. This example uses a specific version for each libray, but in general the latest major GA version of each package is suggested. Note that the requirments.txt file is required in docker serving container in later steps.

In [ ]:
%%writefile requirements.txt
joblib==1.1.0
google-cloud-storage==2.4.0
google-cloud-aiplatform==1.16.0
google-cloud-bigquery==2.34.4
numpy
pandas
plotly==5.9.0
pyarrow==8.0.0
fastapi~=0.63
pystan
prophet==1.1

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"
    
# install dependencies from requirements.txt
! pip3 install {USER_FLAG} -r requirements.txt

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

1. [Enable the Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=compute.googleapis.com).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` or `%` as shell commands, and it interpolates Python variables with `$` or `{}` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [3]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [4]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [5]:
import string
import random

# Generate a uuid of length 8
def generate_uuid():
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))

UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench notebooks**, your environment is already
authenticated.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [6]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

To update your model artifacts without re-building the container, you must upload your model
artifacts and any custom code to Cloud Storage.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

In [7]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = "gs://{}".format(BUCKET_NAME)

In [8]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + UUID

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [9]:
! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [10]:
! gsutil ls -al $BUCKET_URI

### Create a directory for all your model artifacts

In [11]:
%mkdir -p app

### Configure model, repository, and artifact names

Configure the following parameters for the model :

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that is pushed.

`MODEL_NAME` - Name of the Model.

`MODEL_DISPLAY_NAME` - Display name of the Model.

In [12]:
MODEL_ARTIFACT_DIR = "[your-model-artifact-dir-name]"  # @param {type:"string"}
REPOSITORY = "[your-repository-name]"  # @param {type:"string"}
IMAGE = "[your-image-name]"  # @param {type:"string"}
MODEL_NAME = "[your-model-name]"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

In [13]:
# Otherwise, choose deafult names
if MODEL_ARTIFACT_DIR == "[your-model-artifact-dir-name]":
    MODEL_ARTIFACT_DIR = "custom-fb-container-prediction-model"
if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-fb-container-prediction"
if IMAGE == "[your-image-name]":
    IMAGE = "custom-fb-fastapi-server"
if MODEL_NAME == "[your-model-name]":
    MODEL_NAME = "fb_custom_container"
if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "FB Prophet Forecast"

### Import libraries and define constants

In [14]:
import datetime
import os

# BQ libraries
import google.auth
from prophet import Prophet
from google.cloud import bigquery

TODAY = datetime.date.today()
credentials, _ = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
bqclient = bigquery.Client(
    credentials=credentials,
    project=PROJECT_ID,
)

## Train and store the model locally
Next, you define functions to train time-series models using fbprophet for a few sales categories. 

There are two types of forecast functions that are going to be defined. One that only uses sales, and the other takes the [weather variable as an additional regressor](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html) for forecasting.

Weather data comes from a public dataset in BigQuery which is described [here](
https://cloud.google.com/blog/products/gcp/global-historical-daily-weather-data-now-available-in-bigquery).

At the end, the functions export your trained model as a (`.sav`) file using Joblib.

### Define train function for sales model

In the next cell, you define a function that fetches the liquor sales data from BigQuery, processes it and trains and saves the Prophet model.

In [15]:
import joblib


def train(category="1012100"):
    # Download query results. # change to Data, Adj_Close
    query_string = """
    SELECT date as Date, upper(category_name) as category_name, category, sum(sale_dollars) as sales
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
    where category = '{}'
      and date >= '2020-01-01'
    group by Date, category_name, category
    """.format(
        category
    )

    data = bqclient.query(query_string).result().to_dataframe()
    data.plot(title=f"{category} Daily Sales", x="Date", y="sales")

    # Copy results into a dataframe with two columns ds=Date, y=value
    df_forecast = data.copy()
    df_forecast.reset_index(inplace=True)
    df_forecast["ds"] = df_forecast["Date"]
    df_forecast["y"] = df_forecast[
        "sales"
    ]  # Add underscore as BQ changes this field name
    df_forecast = df_forecast[["ds", "y"]]

    # Train the Prophet model
    model = Prophet()
    model.fit(df_forecast)

    # Save the model locally
    joblib.dump(model, "{}.sav".format(category))

#### Train a model for two sample categories (Canadian Whiskies and American Vodkas)

This model uses only the Iowa liquor sales data as input.

In [16]:
%cd app/

# Choose a few product categories to build forecasts for from the Iowa Liquor sales data in BQ: bigquery-public-data:iowa_liquor_sales.sales
# 1012100 CANADIAN WHISKIES
# 1031100 AMERICAN VODKAS
train("1012100")
train("1031100")

%cd ..

### Define train function for sales and weather model
The query for the regressor retrieves 2 years of weather data, and uses a moving average to generate 100 future weather data points.

In [30]:
def train_w_regressor(category="1012100"):
    # Download query results. # change to Data, Adj_Close
    query_string = """
    SELECT date as Date, upper(category_name) as category_name, category, sum(sale_dollars) as sales
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
    where category = '{CATEGORY}'
      and date >= '2020-01-01'
    group by Date, category_name, category
    """.format(
        CATEGORY=category
    )

    query_string_regressor = """
    WITH history as
    (
    SELECT
      wx.date as ds,
      AVG(wx.value/10) AS temp
    FROM
      `bigquery-public-data.ghcn_d.ghcnd_stations` AS stn
    JOIN
      `bigquery-public-data.ghcn_d.ghcnd_2021` AS wx ON wx.id = stn.id
    WHERE
      stn.state = 'AZ'
      AND wx.element = 'TMIN'
      AND wx.qflag IS NULL
    GROUP by wx.date
    union all
    SELECT
      wx.date as ds,
      AVG(wx.value/10) AS temp
    FROM
      `bigquery-public-data.ghcn_d.ghcnd_stations` AS stn
    JOIN
      `bigquery-public-data.ghcn_d.ghcnd_2020` AS wx ON wx.id = stn.id
    WHERE
      stn.state = 'AZ'
      AND wx.element = 'TMIN'
      AND wx.qflag IS NULL
    GROUP by wx.date
    )
    ,next_100_days as
    (
    SELECT DATE_ADD(max(stn.date) over (order by 1), INTERVAL row_number() over (order by stn.id) DAY) as ds, null as temp
    FROM `bigquery-public-data.ghcn_d.ghcnd_2021` AS stn
    LIMIT 100
    )
    ,combined_data as
    (
    select *
    from history
    union all
    select * from next_100_days
    )
    select c.ds
          ,case when temp is null then AVG(c.temp) OVER (ORDER BY c.ds ASC ROWS 100 PRECEDING)
           else temp end as temp
    from combined_data c;
    """

    data = bqclient.query(query_string).result().to_dataframe()
    data.plot(title=f"{category} Daily Sales", x="Date", y="sales")

    # Copy results into a dataframe with two columns ds=Date, y=value
    df_forecast = data.copy()
    df_forecast.reset_index(inplace=True)
    df_forecast["ds"] = df_forecast["Date"]
    df_forecast["y"] = df_forecast["sales"]  # Add underscore as BQ changes this field name
    df_forecast = df_forecast[["ds", "y"]]

    # Return the weather data from BQ
    data_regressor = bqclient.query(query_string_regressor).result().to_dataframe()
    df_regressor = data_regressor.copy()

    # Deal with any NaN values using fillna
    df_regressor_out = df_regressor.fillna(method="pad")
    df_regressor_out.to_csv(f"{category}_weather.csv", index=False)

    # Add a temp column to the df_forecast dataframe, and populate it from the df_regressor dataframe
    df_forecast["temp"] = df_forecast.ds.map(
        df_regressor.set_index("ds")["temp"].to_dict()
    )
    
    df_forecast.loc[:, "temp"] = df_forecast.fillna(method="pad").fillna(
        method="backfill"
    )  # Address NaN values of temp
    
    # Train the Prophet model with the regressor
    model = Prophet()
    model = model.add_regressor("temp")
    model.fit(df_forecast)

    # Save the model locally, with _weather appended to the file name
    joblib.dump(model, f"{category}_weather.sav")

#### Train a model with weather data for the same sample categories (Canadian Whiskies and American Vodkas)

This model uses only the Iowa liquor sales data, and an additional regressor to account for the impact of weather. As mentioned before, the regressor must contain for both the history and future dates. In the query that fetches the regressor data, a moving average is used to calculate the future weather data.

More information on regressors can be found [here](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html).

In [31]:
%cd app/

# Choose a few product categories to build forecasts for from the Iowa Liquor sales data in BQ: bigquery-public-data:iowa_liquor_sales.sales
# 1012100 CANADIAN WHISKIES
# 1031100 AMERICAN VODKAS
train_w_regressor("1012100")
train_w_regressor("1031100")

%cd ..

### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in the Cloud Storage:

* `{category}.sav` (model artifact) for each sales category that you would like to retrieve the forecast for.
* `*.csv` - Data for the weather regressor that is needed at prediction time.

Run the following cell to upload your files:

In [32]:
%cd app
!gsutil cp *.sav *.csv {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
%cd ..

## Local model testing

Review the trained FB Prophet models with and without the regressor in the sections below.

Check the set environment variables.

In [33]:
os.environ["AIP_STORAGE_URI"] = f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}"
os.environ["AIP_HTTP_PORT"] = "8080"
os.environ["AIP_HEALTH_ROUTE"] = "/health"
os.environ["AIP_PREDICT_ROUTE"] = "/predict"

In [34]:
print(os.environ["AIP_STORAGE_URI"])
print(os.environ["AIP_HTTP_PORT"])
print(os.environ["AIP_HEALTH_ROUTE"])
print(os.environ["AIP_PREDICT_ROUTE"])

### **Test the sales model**

In [35]:
%cd app

import datetime

import pandas as pd
from google.cloud import storage

gcs_client = storage.Client(project=PROJECT_ID)

# Set a sample category and number of days to test with
category = "1012100"
days = 7

# Test the loading of the model from GCS
fname = f"{category}.sav"
with open(f"{fname}", "wb") as model_g:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/{fname}", model_g
    )

model = joblib.load(f"{category}.sav")

# Create a dataframe that ranges from 2020 to Today + the number of days set above
TODAY = datetime.date.today()
future = TODAY + datetime.timedelta(days=days)

dates = pd.date_range(
    start="2020-01-01",
    end=future.strftime("%m/%d/%Y"),
)
df = pd.DataFrame({"ds": dates})

# Run a prediction for these dates and save it to a forecast dataframe
forecast = model.predict(df)

model.plot(forecast).savefig(f"{category}_plot.png")
model.plot_components(forecast).savefig(f"{category}_plot_components.png")

forecast.tail(days).to_dict("records")

%cd ..

### **Test the sales and weather model**

In [36]:
%cd app

gcs_client = storage.Client(project=PROJECT_ID)

# Set a sample category and number of days to test with
category = "1012100_weather"
days = 7

# Test the loading of the model from GCS
fname = f"{category}.sav"
with open(f"{fname}", "wb") as model_g:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/{fname}", model_g
    )

model = joblib.load(f"{category}.sav")

# Test the loading of the regressor from GCS
fname_csv = f"{category}.csv"
with open(f"{fname_csv}", "wb") as model_csv:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/{fname_csv}", model_csv
    )

df_regressor = pd.read_csv(fname_csv)

# Return the data from the regressor that ranges from 2020 to Today + the number of days set above
TODAY = datetime.date.today()
future = TODAY + datetime.timedelta(days=days)

start = "2020-01-01"
after_start_date = df_regressor["ds"] >= start
before_end_date = df["ds"] <= future.strftime("%m/%d/%Y")
between_two_dates = after_start_date & before_end_date
df_final = df_regressor.loc[between_two_dates]
df_final.tail()

# Run a prediction for these dates and the regressor
forecast = model.predict(df_final)

model.plot(forecast).savefig(f"{category}_plot.png")
model.plot_components(forecast).savefig(f"{category}_plot_components.png")

%cd ..

## Build a FastAPI server

To serve predictions from both the models, build a FastAPI server application.

In [37]:
%%writefile app/main.py
from fastapi import FastAPI, Request
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse

import joblib
import json
import numpy as np
import os
import datetime
import pandas as pd
import errno

from google.cloud import storage

app = FastAPI()
gcs_client = storage.Client()

@app.exception_handler(Exception)
async def validation_exception_handler(request, err):
    base_error_message = f"Failed to execute: {request.method}: {request.url}"
    # Change here to LOGGER
    return JSONResponse(status_code=400, content={"message": f"{base_error_message}. Detail: {err}"})

    
def download_model(category="1012100", regressor=""): #defaults to "1012100"
    if not category:
        raise HTTPException(status_code=400, detail=f"category not found. category={category}")
    fname = f"{category}{regressor}.sav"
    with open(f"{fname}", 'wb') as model_g:
        gcs_client.download_blob_to_file(
            f"{os.environ['AIP_STORAGE_URI']}/{fname}", model_g
        )
    model = joblib.load (fname)
    return model
    
def predict_in(model, days=7, regressor="", category="1012100"):
    TODAY = datetime.date.today()
    future = TODAY + datetime.timedelta(days=days)

    dates = pd.date_range(start="2020-01-01", end=future.strftime("%m/%d/%Y"),)
    df = pd.DataFrame({"ds": dates})

    if (regressor > ""):
        fname_csv = f"{category}{regressor}.csv"
        with open(f"{fname_csv}", 'wb') as model_csv:
            gcs_client.download_blob_to_file(
                f"{os.environ['AIP_STORAGE_URI']}/{fname_csv}", model_csv
            )

        df_regressor = pd.read_csv(fname_csv)
        start="2020-01-01"
        after_start_date = df_regressor["ds"] >= start
        before_end_date = df["ds"] <= future.strftime("%m/%d/%Y")
        between_two_dates = after_start_date & before_end_date
        df_final = df_regressor.loc[between_two_dates]
        df_final.tail()
        
        forecast = model.predict(df_final)
    
    else:
        forecast = model.predict(df)

    model.plot(forecast).savefig("model_plot.png")
    model.plot_components(forecast).savefig("model_plot_components.png")    
    
    return forecast.tail(days).to_dict("records")


def convert(prediction_list):
    output = {}
    for data in prediction_list:
        date = data["ds"].strftime("%m/%d/%Y")
        output[date] = data["trend"]
    return output


@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    try:
        if type(body) is list:
            body = body[0]
    except:
        None #Do Nothing
    print (body)
    instances = body["instances"]

    try:
        if type(instances) is list:
            instances = instances[0]
    except:
        None #Do Nothing
    print(instances)    
    category = instances["category"] 
    days = instances["days"]
    regressor = instances["regressor"]
    
    try:
        if type(category) is list:
            category = category[0]
    except:
        None #Do Nothing
    print(category) 

    try:
        if type(days) is list:
            days = days[0]
    except:
        None #Do Nothing
    print(days) 

    try:
        if type(regressor) is list:
            regressor = regressor[0]
    except:
        None #Do Nothing
    print(regressor) 
    
    model_download = download_model (category, regressor)
    prediction_list = predict_in(model_download, days, regressor, category)  
    
    if not prediction_list:
        raise HTTPException(status_code=400, detail="Model not found.")
    
    prediction_output = convert(prediction_list)
    final_output = [(k, v) for k, v in prediction_output.items()] 
    
    return {"predictions": final_output}

### Add pre-start script
FastAPI executes this script before starting up the server. The `PORT` environment variable is set to equal `AIP_HTTP_PORT` in order to run FastAPI on same the port expected by Vertex AI Predictions.

In [38]:
%%writefile app/prestart.sh
export PORT=$AIP_HTTP_PORT

### Create test instances
To learn more about formatting input instances in JSON, [read the documentation.](https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#request-body-details)

In [39]:
%%writefile instances.json
{"instances": [{"category":["1012100"], "days": [30], "regressor": ["_weather"]}]}

## Build and push container to Artifact Registry

Write the `Dockerfile` using `tiangolo/uvicorn-gunicorn-fastapi` as a base image. This automatically runs FastAPI for you using Gunicorn and Uvicorn. Visit [the FastAPI docs to read more about deploying FastAPI with Docker](https://fastapi.tiangolo.com/deployment/docker/) to learn more.

In [40]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

In [41]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

Build the image tagging the destination-Artifact Registry path. The logs are stored in `logs.txt`.

In [42]:
def build_image():
    ! sudo docker build --tag="{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}" . > logs.txt


if not IS_COLAB:
    print("Building the image...")
    build_image()
    print ("Process completed !")

### Run and test the container locally (optional)

Test running the container locally in detached mode and provide the environment variables that the container needs. These environment variables are provided to the container by Vertex AI Predictions once deployed on Vertex AI. Test the `/health` and `/predict` routes and then stop the running image.

**Note**: This section tests the container locally. However, if the underlying environment is Colab, the `docker` commands are automatically skipped.

In [43]:
if not IS_COLAB:
    !sudo docker stop local-ts
    !sudo docker rm local-ts
    !sudo docker run -d -p 80:8080 \
        --name=local-ts \
        -e AIP_HTTP_PORT=8080 \
        -e AIP_HEALTH_ROUTE=/health \
        -e AIP_PREDICT_ROUTE=/predict \
        -e AIP_STORAGE_URI={BUCKET_URI}/{MODEL_ARTIFACT_DIR} \
        "{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}"

Ping the health route.

In [44]:
if not IS_COLAB:
    !curl localhost/health

#### Test the predict route

Pass the `instances.json` and test the predict route.

In [45]:
if not IS_COLAB:
    !curl -X POST \
      -d @instances.json \
      -H "Content-Type: application/json; charset=utf-8" \
      localhost/predict

Now, store an instance without the `regressor` and test the predict route. This retreives forecasts from the model trained without the weather regressor variable.

In [46]:
%%writefile instances.json
{"instances": [{"category":["1012100"], "days": [30], "regressor": [""]}]}

In [47]:
if not IS_COLAB:
    !curl -X POST \
      -d @instances.json \
      -H "Content-Type: application/json; charset=utf-8" \
      localhost/predict

Stop the container locally.

In [48]:
if not IS_COLAB:
    !sudo docker stop local-ts

### Push the container to artifact registry

Configure Docker to access Artifact Registry. Then push your container image to your Artifact Registry repository.

#### Enable the Artifact Registry API
Make sure to enable the Artifact Registry API here if you haven't already enabled.

In [49]:
!gcloud services enable artifactregistry.googleapis.com

Run this below cell once to create the artifact repository.

In [50]:
!gcloud beta artifacts repositories create {REPOSITORY} \
 --repository-format=docker \
 --location=$REGION

Push the image to the created artifact repository using Cloud-Build.

In [51]:
!gcloud builds submit --region={REGION} --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

## Deploy to Vertex AI

### Create Vertex AI model using artifact uri
Use the Python SDK to upload and deploy your model from the artifact registry.

In [52]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
)

### Deploy the model to endpoint

Deploy the model to an endpoint. After this step completes, the model is deployed and ready for online predictions.

In [53]:
endpoint = model.deploy(machine_type="n1-standard-4")

## Request predictions

Send online requests to the model deployed to the endpoint and get predictions.

### Using Python SDK

Get predictions from the endpoint using python SDK.

In [54]:
# Send an instance for the model without regressor
endpoint.predict(instances=[{"category": ["1012100"], "days": [30], "regressor": [""]}])

In [55]:
# Send an instance for the model with regressor
endpoint.predict(
    instances=[{"category": ["1012100"], "days": [30], "regressor": ["_weather"]}]
)

### Using REST

Get predictions from the endpoint using curl request.

In [56]:
ENDPOINT_ID = endpoint.name

In [57]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

### Using gcloud CLI

Get predictions from the endpoint using gcloud CLI.

In [58]:
!gcloud beta ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

## Cleaning up

To clean up all the Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [59]:
# Undeploy model and delete endpoint
endpoint.undeploy_all()
endpoint.delete()

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

Set the `delete_bucket` to **True** to delete the Cloud Storage bucket.

Set the `delete_art_repo` to **True** to delete the Artifact Registry.

In [60]:
delete_bucket = False
delete_art_repo = False

# Delete the Cloud Storage bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

# Delete the artifact registry
if delete_art_repo or os.getenv("IS_TESTING"):
    ! gcloud beta artifacts repositories delete {REPOSITORY} --location=$REGION -q